In [1]:
import pandas as pd

Читаем данные и получаем информацию о том, какие данные находятся в файлах, размер данных

In [2]:
clust = pd.read_csv('address_clust.csv')
stats = pd.read_csv('address_stats.csv')

Для dataframe clust

In [3]:
clust.head()

,address_id,cluster_id
0,71413451,1
1,71411914,1
2,71410369,1
3,71410100,1
4,71410040,1


In [4]:
clust.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6996 entries, 0 to 6995
Data columns (total 2 columns):
address_id    6996 non-null int64
cluster_id    6996 non-null int64
dtypes: int64(2)
memory usage: 109.4 KB


In [6]:
clust.describe()

,address_id,cluster_id
count,6.996000e+03,6996.000000
mean,5.725031e+07,1.477987
std,5.566587e+06,0.499551
min,4.652909e+07,1.000000
25%,5.314442e+07,1.000000
50%,5.547587e+07,1.000000
75%,5.695893e+07,2.000000
max,7.141345e+07,2.000000


In [7]:
clust['address_id'].nunique()

6996

Для dataframe stats

In [8]:
stats.head()

,id,address_id,transaction_id,received,sent
0,87134765,46402336,19162323,0.0,1800000.0
1,87134766,45919235,19162323,0.0,1071052.0
2,87134767,46529090,19162323,1000000.0,0.0
3,87134768,46529091,19162323,1821052.0,0.0
4,87154706,46529090,19166856,0.0,1000000.0


In [9]:
stats.shape

(76724, 5)

In [10]:
stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76724 entries, 0 to 76723
Data columns (total 5 columns):
id                76724 non-null int64
address_id        76724 non-null int64
transaction_id    76724 non-null int64
received          76724 non-null float64
sent              76724 non-null float64
dtypes: float64(2), int64(3)
memory usage: 2.9 MB


In [11]:
stats['address_id'].nunique()

12208

In [12]:
stats['transaction_id'].nunique()

13753

Объеденим таблицы clust и stats в таблицу new_stats и посмотрим что она будет содержать

In [13]:
new_stats = pd.merge(left=clust,right=stats, how='outer', left_on=['address_id'], right_on=['address_id'])

In [14]:
new_stats.head()

,address_id,cluster_id,id,transaction_id,received,sent
0,71413451,1.0,135571000,29319673,1003590.0,0.0
1,71413451,1.0,135679242,29338157,0.0,1003590.0
2,71411914,1.0,135568134,29319205,1259370.0,0.0
3,71411914,1.0,135679245,29338157,0.0,1259370.0
4,71410369,1.0,135565307,29318455,21009000.0,0.0


In [15]:
new_stats.shape

(76724, 6)

In [16]:
new_stats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76724 entries, 0 to 76723
Data columns (total 6 columns):
address_id        76724 non-null int64
cluster_id        58475 non-null float64
id                76724 non-null int64
transaction_id    76724 non-null int64
received          76724 non-null float64
sent              76724 non-null float64
dtypes: float64(3), int64(3)
memory usage: 4.1 MB


Видно, что количество строк в таблице new_stats осталось таким же как и в таблице stats (76724 строк), a незаполненные значения есть только у cluster_id, для которых можно сопоставить кластеру 0

In [17]:
new_stats.fillna(0, inplace = True)

In [18]:
new_stats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76724 entries, 0 to 76723
Data columns (total 6 columns):
address_id        76724 non-null int64
cluster_id        76724 non-null float64
id                76724 non-null int64
transaction_id    76724 non-null int64
received          76724 non-null float64
sent              76724 non-null float64
dtypes: float64(3), int64(3)
memory usage: 4.1 MB


In [19]:
new_stats.groupby(['cluster_id'])['sent'].sum()

cluster_id
0.0    1.030563e+11
1.0    6.402648e+10
2.0    1.031243e+11
Name: sent, dtype: float64

Сколько биткоинов отправил кластер 1 в кластер 2 и сколько кластер 2 отправил в кластер 1?

Функция, которая будет вычислять сколько один кластер отправил биткоинов другому кластеру:

In [20]:
def clust_send(sent_id, recived_id):
    k = 10**8
    trans_id = new_stats[(new_stats['cluster_id'] == sent_id) & (new_stats['sent'] > 0)]['transaction_id'].unique()
    sum_satochi = new_stats[(new_stats['transaction_id'].isin(trans_id)) & 
                            (new_stats['cluster_id'] == recived_id)]['received'].sum()
    x = sum_satochi/k
    return print('Кластер {0} отпривил в кластер {1}: {2} биткоинов'.format(sent_id, recived_id, x))

In [21]:
sent_id = 1
recived_id = 2
clust_send(sent_id, recived_id)

Кластер 1 отпривил в кластер 2: 129.24324 биткоинов


In [22]:
sent_id = 2
recived_id = 1
clust_send(sent_id, recived_id)

Кластер 2 отпривил в кластер 1: 135.82 биткоинов


Сколько биткоинов отправил кластер 1 в кластер 0 и сколько кластер 0 отправил в кластер 1?

In [23]:
sent_id = 1
recived_id = 0
clust_send(sent_id, recived_id)

Кластер 1 отпривил в кластер 0: 300.55392888 биткоинов


In [24]:
sent_id = 0
recived_id = 1
clust_send(sent_id, recived_id)

Кластер 0 отпривил в кластер 1: 297.11968388 биткоинов


Сколько биткоинов отправил кластер 2 в кластер 0 и сколько кластер 0 отправил в кластер 2?

In [25]:
sent_id = 2
recived_id = 0
clust_send(sent_id, recived_id)

Кластер 2 отпривил в кластер 0: 4.86984 биткоинов


In [26]:
sent_id = 0
recived_id = 2
clust_send(sent_id, recived_id)

Кластер 0 отпривил в кластер 2: 9.74013861 биткоинов


Сколько биткоинов потратил кластер 1 и кластер 2 на fee?

Функция, которая будет вычислять сколько кластер потратил на fee:

In [27]:
def clust_sent_fee(sent_id):
    k = 10**8
    trans_id = new_stats[(new_stats['cluster_id'] == sent_id) & (new_stats['sent'] > 0)]['transaction_id'].unique()
    clust_sent_fee = new_stats[(new_stats['cluster_id'] == sent_id)]['sent'].sum()
    clust_sent = new_stats[(new_stats['transaction_id'].isin(trans_id)) & 
                           (new_stats['cluster_id'].isin([0, 1, 2]))]['received'].sum()
    fee = (clust_sent_fee - clust_sent)/k
    return print('Кластер {0} потратил на fee: {1} биткоинов'.format(sent_id, fee))

In [28]:
sent_id = 1
clust_sent_fee(sent_id)

Кластер 1 потратил на fee: 0.667144 биткоинов


In [29]:
sent_id = 2
clust_sent_fee(sent_id)

Кластер 2 потратил на fee: 0.74566722 биткоинов
